In [7]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

In [16]:
df=pd.read_csv("../moreCleanData.csv", index_col=False)
df.head()

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,PAPERLESSBILLING,MONTHLYCHARGES,TOTALCHARGES,MONTHLY_MINUTES_OF_USE,...,STREAMINGMOVIES_No,STREAMINGMOVIES_No internet service,STREAMINGMOVIES_Yes,CONTRACT_Month-to-month,CONTRACT_One year,CONTRACT_Two year,PAYMENTMETHOD_Bank transfer automatic,PAYMENTMETHOD_Credit card automatic,PAYMENTMETHOD_Electronic check,PAYMENTMETHOD_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,0,...,1,0,0,1,0,0,0,0,1,0
1,1,0,0,0,45,0,0,42.30,1840.75,0,...,1,0,0,0,1,0,1,0,0,0
2,0,0,0,0,10,0,0,29.75,301.90,0,...,1,0,0,1,0,0,0,0,0,1
3,1,1,0,0,1,0,1,39.65,39.65,0,...,0,0,1,1,0,0,0,0,1,0
4,1,0,1,1,1,0,0,30.20,30.20,0,...,1,0,0,1,0,0,0,0,1,0


In [9]:
#Prepare target & features for modeling
cols = df.columns
train_cols = cols.drop(["CHURN"])
features = df[train_cols]
target = df["CHURN"]
train_cols

Index(['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'PAPERLESSBILLING', 'MONTHLYCHARGES', 'TOTALCHARGES',
       'MONTHLY_MINUTES_OF_USE', 'TOTAL_MINUTES_OF_USE', 'MONTHLY_SMS',
       'TOTAL_SMS', 'MULTIPLELINES_No', 'MULTIPLELINES_No phone service',
       'MULTIPLELINES_Yes', 'INTERNETSERVICE_DSL',
       'INTERNETSERVICE_Fiber optic', 'INTERNETSERVICE_No',
       'ONLINESECURITY_No', 'ONLINESECURITY_No internet service',
       'ONLINESECURITY_Yes', 'ONLINEBACKUP_No',
       'ONLINEBACKUP_No internet service', 'ONLINEBACKUP_Yes',
       'DEVICEPROTECTION_No', 'DEVICEPROTECTION_No internet service',
       'DEVICEPROTECTION_Yes', 'TECHSUPPORT_No',
       'TECHSUPPORT_No internet service', 'TECHSUPPORT_Yes', 'STREAMINGTV_No',
       'STREAMINGTV_No internet service', 'STREAMINGTV_Yes',
       'STREAMINGMOVIES_No', 'STREAMINGMOVIES_No internet service',
       'STREAMINGMOVIES_Yes', 'CONTRACT_Month-to-month', 'CONTRACT_One year',
       'CONTRAC

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
#rf.fit(features, target)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["CHURN"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["CHURN"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["CHURN"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["CHURN"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {target.values[:10].tolist()}")
### END SOLUTION

1.0
0.0
First 10 Predictions:   [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
First 10 Actual labels: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


## Decision tree

In [11]:
from sklearn import tree
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
clf.score(X_test, y_test)

1.0

In [12]:
# False positives.
fp_filter = (predictions == 1) & (y_test == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (y_test == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (y_test == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (y_test == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)

1.0
0.0


In [13]:
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1, 0, 0, 0, 0, 0, 1, 0, 1, 1]
First 10 Actual labels: [1, 0, 0, 0, 0, 0, 1, 0, 1, 1]


In [14]:
df.shape

(119731, 45)

In [17]:
df["CHURN"]

0         0
1         0
2         0
3         1
4         1
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        1
15        1
16        1
17        1
18        1
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        1
28        0
29        0
         ..
119701    1
119702    1
119703    0
119704    0
119705    0
119706    0
119707    0
119708    0
119709    1
119710    0
119711    0
119712    1
119713    0
119714    0
119715    0
119716    0
119717    1
119718    0
119719    0
119720    0
119721    0
119722    1
119723    0
119724    1
119725    0
119726    0
119727    0
119728    0
119729    1
119730    0
Name: CHURN, Length: 119731, dtype: int64

In [22]:
#Prepare target & features for modeling
cols = df.columns
train_cols = cols.drop(["CHURN"])
features = df[['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'PAPERLESSBILLING', 'MONTHLYCHARGES', 'TOTALCHARGES',
       'MONTHLY_MINUTES_OF_USE', 'TOTAL_MINUTES_OF_USE', 'MONTHLY_SMS',
       'TOTAL_SMS', 'MULTIPLELINES_No', 'MULTIPLELINES_No phone service',
       'MULTIPLELINES_Yes', 'INTERNETSERVICE_DSL',
       'INTERNETSERVICE_Fiber optic', 'INTERNETSERVICE_No',
       'ONLINESECURITY_No', 'ONLINESECURITY_No internet service',
       'ONLINESECURITY_Yes', 'ONLINEBACKUP_No',
       'ONLINEBACKUP_No internet service', 'ONLINEBACKUP_Yes',
       'DEVICEPROTECTION_No', 'DEVICEPROTECTION_No internet service',
       'DEVICEPROTECTION_Yes', 'TECHSUPPORT_No']]
target = df["CHURN"]
train_cols

Index(['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'PAPERLESSBILLING', 'MONTHLYCHARGES', 'TOTALCHARGES',
       'MONTHLY_MINUTES_OF_USE', 'TOTAL_MINUTES_OF_USE', 'MONTHLY_SMS',
       'TOTAL_SMS', 'MULTIPLELINES_No', 'MULTIPLELINES_No phone service',
       'MULTIPLELINES_Yes', 'INTERNETSERVICE_DSL',
       'INTERNETSERVICE_Fiber optic', 'INTERNETSERVICE_No',
       'ONLINESECURITY_No', 'ONLINESECURITY_No internet service',
       'ONLINESECURITY_Yes', 'ONLINEBACKUP_No',
       'ONLINEBACKUP_No internet service', 'ONLINEBACKUP_Yes',
       'DEVICEPROTECTION_No', 'DEVICEPROTECTION_No internet service',
       'DEVICEPROTECTION_Yes', 'TECHSUPPORT_No',
       'TECHSUPPORT_No internet service', 'TECHSUPPORT_Yes', 'STREAMINGTV_No',
       'STREAMINGTV_No internet service', 'STREAMINGTV_Yes',
       'STREAMINGMOVIES_No', 'STREAMINGMOVIES_No internet service',
       'STREAMINGMOVIES_Yes', 'CONTRACT_Month-to-month', 'CONTRACT_One year',
       'CONTRAC

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
#rf.fit(features, target)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["CHURN"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["CHURN"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["CHURN"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["CHURN"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {target.values[:10].tolist()}")
### END SOLUTION

1.0
0.0
First 10 Predictions:   [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
First 10 Actual labels: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


In [25]:
#Prepare target & features for modeling
cols = df.columns
train_cols = cols.drop(["CHURN"])
features = df[['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'PAPERLESSBILLING', 'MONTHLYCHARGES', 'TOTALCHARGES',
       'MONTHLY_MINUTES_OF_USE', 'TOTAL_MINUTES_OF_USE', 'MONTHLY_SMS',
       'TOTAL_SMS', 'MULTIPLELINES_No', 'MULTIPLELINES_No phone service',
       'MULTIPLELINES_Yes', 'INTERNETSERVICE_DSL',
       'INTERNETSERVICE_Fiber optic', 'INTERNETSERVICE_No',
       'ONLINESECURITY_No', 'ONLINESECURITY_No internet service',
       'ONLINESECURITY_Yes']]
target = df["CHURN"]

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
#rf.fit(features, target)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["CHURN"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["CHURN"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["CHURN"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["CHURN"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {target.values[:10].tolist()}")
### END SOLUTION

1.0
0.0
First 10 Predictions:   [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
First 10 Actual labels: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


In [51]:
#Prepare target & features for modeling
cols = df.columns
train_cols = cols.drop(["CHURN"])
features = df[['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE','MONTHLYCHARGES'
       ]]
target = df["CHURN"]

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
#rf.fit(features, target)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["CHURN"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["CHURN"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["CHURN"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["CHURN"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {target.values[:10].tolist()}")
### END SOLUTION

0.9983948635634029
0.011664658132290411
First 10 Predictions:   [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
First 10 Actual labels: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
